**Coin-Row problem**: Given a row of `n` coins, with values `[c1, c2, ..., cn]`, the goal is to pick up the subset of coins with the largest combined value subject to the constraint that we cannot pick any two adjacent coins.

Let `F(n)` be defines as the largest combined value given a row of `n` coins.

Using the dynamic programming approach, we consider two possible solutions:

(1) The solution subset includes the `nth` coin => it cannot contain the `(n-1)th` coin such that the remaining coins must be picked from the first `(n-2)` and the solution can be expressed as:

`F(n) = cn + F(n-2)`

(2) The solution subset does not contain the nth coin => the coins that can be picked must come from the first `(n-1)` coins, so the solution can be expressed as:

`F(n) = F(n-1)`


Then the final solution must be the larger of these two possibilities:

`F(n) = max(F(n-1), cn + F(n-2))`

which is a `recurrance relation` with base case `F(0) = 0, F(1) = c1`


In [3]:
import numpy as np

def coin_row(C):
    # create an array to store solutions for subproblems
    F = np.zeros(shape=(len(C)+1))

    # base cases
    F[0] = 0
    F[1] = C[0]

    # coins pick for each subproblem solution
    coins_picked = [[], [C[0]]]
    
    # bottom up solution for finding F[n-1]
    for i in range(2,len(C)+1):
        F[i] = max(F[i-1],  C[i-1]+F[i-2])
        
        if(C[i-1]+F[i-2] > F[i-1]):
            coins_picked.append([C[i-1]]+coins_picked[i-2])
        else:
            coins_picked.append(coins_picked[i-1])   

    return F, coins_picked

In [4]:

# row of 6 coins with the following values
C = np.array([5.,1.,2.,10.,6.,2.])
F, coins_picked = coin_row(C)

print(f"Coin values: {C}")
print(f"Subproblems solutions: {F}")
print(f"Subproblems coins_picked: {coins_picked}")



Coin values: [ 5.  1.  2. 10.  6.  2.]
Subproblems solutions: [ 0.  5.  5.  7. 15. 15. 17.]
Subproblems coins_picked: [[], [5.0], [5.0], [2.0, 5.0], [10.0, 5.0], [10.0, 5.0], [2.0, 10.0, 5.0]]


**Change-Making Problem**: We want to change a bill of value `N` into `m` smaller bill denominations `d1< d2 < d3 < ... < dm` (where `d1 = 1`) subject to the constraint that we want to minimize the total number of smaller bills used.

Let `F(n)` be the minimum number of bills ading to give a total value `n`.

To solve this problem using dynamic programming, first consider the set of values `{n-d1, n-d2, ...., n-dj}` where `dj <= n` and the corresponding set of minimum number of bills required to change these values are `{F(n-d1), F(n-d2), ..., F(n-dj)}`. Then if we take the minimum value from this set, say `F(n-dj)` and add one bill of `dj` to this, we get our desired solution for `F(n)` which can be expressed as:

`F(n) = 1 + min_(j: dj<=n) { F(n-dj) }`

This is a recurrance relation with base case `F(0) = 0`.

To solve this problem in a bottom-up fashion, we can compute all subponblem solutions `F(n),  n = 0, 1, ...N` starting with `n = 1` and store them in an array


In [5]:
def change_making(d, N):

    # array of subproblem solutions
    F = np.zeros(shape=(N+1))

    # bills picked for each subproblems
    bills_picked = [[]]

    # compute solutions for all n = 1,2, ...N
    for i in range(1,N+1):

        # find smallest F(n-dj) for all dj <=  n
        minF = 1e25 # ~infinity
        dj_min = d[0]
        for dj in d:
            if (dj > i):
                break
            if(F[i-dj] < minF):
                minF = F[i-dj]
                dj_min = dj      
   
        F[i] = 1 + minF 
        bills_picked.append([dj_min] + bills_picked[i-dj_min])
    
    return F, bills_picked   

In [6]:
d = np.array([1,2,5,10])
N = 20

F, bills_picked = change_making(d,N)
print(f"N = {N}, bill denominations = {d}")
print(f"n   | F(n)  | bills_picked ")
for i in range(len(F)):
    print(f"{i}     {F[i]}     {bills_picked[i]}")



N = 20, bill denominations = [ 1  2  5 10]
n   | F(n)  | bills_picked 
0     0.0     []
1     1.0     [1]
2     1.0     [2]
3     2.0     [1, 2]
4     2.0     [2, 2]
5     1.0     [5]
6     2.0     [1, 5]
7     2.0     [2, 5]
8     3.0     [1, 2, 5]
9     3.0     [2, 2, 5]
10     1.0     [10]
11     2.0     [1, 10]
12     2.0     [2, 10]
13     3.0     [1, 2, 10]
14     3.0     [2, 2, 10]
15     2.0     [5, 10]
16     3.0     [1, 5, 10]
17     3.0     [2, 5, 10]
18     4.0     [1, 2, 5, 10]
19     4.0     [2, 2, 5, 10]
20     2.0     [10, 10]


**Knapsack Problem**: Given `n` items of values `v1, v2, ..., vn` and weights  `w1, w2, ..., wn` and a knapsack of capacity `W`, the goal is to pick a subset of items that maximizes the total value while the total weight does not exceed the knapsack capacity.

Let `F(i,j)` be the maximum value of a subset of the first `i` items that can fit inside a knapsack of capacity `j`

To solve for `F(i,j)` using dynamic programming, consider the following two possibilities:

(i) The solution does not contain the `ith` => `F(i,j) = F(i-1, j)`
(ii) The solution contains the `ith` item such that the remaining items must come for the first `(i-1)` items and their combined weight excludes the weight of the `ith` item => `F(i,j) = vi + F(i-1, j-wi)`

Then the solution must be the larger of these two possibilities. This is assuming that the `ith` item will fit in the knapsack of capacity `j` i.e. `wi <= j`, otherwise if it doesn't fit, then the solution will only contain items from the first `(i-1)` items => `F(i,j) = F(i-1, j)` if `wi > j`

So we have the follwing recurrance relation:

`F(i,j) = max{ F(i-1,j), vi+F(i-1, j-wi) }`, if `wi <= j`
        
OR

`F(i,j) = F(i-1, j)`,                      if `wi > j`

with base cases `F(0, j) = F(i, 0) = 0` for all `i,j`

We can implement a solution by creating an array for `F(i,j)` and computing all values starting with `i = j = 1`

In [7]:
def knapsack_bottomup(v, w, Wcap):

    # create an array for the subproblem solutions
    F = np.zeros(shape=(len(v)+1, Wcap+1))
    
    # items picked for each subproblem
    items_picked = []
    for i in range(0,len(v)+1):
        b = []
        for j in range(0, Wcap+1):
            b.append([])
        items_picked.append(b)

    for i in range(1,len(v)+1):
        for j in range(1, Wcap+1):
            if(w[i-1] > j):
                F[i,j] = F[i-1, j]
                if(items_picked[i-1][j] != []):
                    items_picked[i][j] += items_picked[i-1][j]
            else:
                F[i,j] = max(F[i-1,j], v[i-1] + F[i-1, j-w[i-1]])   
                if(F[i-1,j] > v[i-1] + F[i-1, j-w[i-1]]):
                    if(items_picked[i-1][j] != []):
                        items_picked[i][j] += items_picked[i-1][j]
                else:
                    items_picked[i][j].append(v[i-1])
                    if(items_picked[i-1][j-w[i-1]] != []):
                        items_picked[i][j] += items_picked[i-1][j-w[i-1]]

    return F[:,:], items_picked    


def knapsack_topdown_recursive(i, j, v, w, F, items_picked):

    # check to make sure that we havent't computed this subproblem solution already
    if(F[i,j] < 0):
        items = []
        if(w[i-1] > j):
            value = knapsack_topdown_recursive(i-1, j, v, w, F, items_picked)
            if(items_picked[i-1][j] != []):
                items += items_picked[i-1][j]
        else:
            val1 = knapsack_topdown_recursive(i-1, j, v, w, F, items_picked)
            val2 = v[i-1] + knapsack_topdown_recursive(i-1, j-w[i-1], v, w, F, items_picked)
            value = max(val1, val2)
                          
            if(val1 > val2):
                if(items_picked[i-1][j] != []):
                    items += items_picked[i-1][j]
            else:
                items = [v[i-1]]
                if(items_picked[i-1][j-w[i-1]] != []):
                    items += items_picked[i-1][j-w[i-1]]
                
        F[i,j] = value
        items_picked[i][j] += items
    return F[i,j]


In [8]:
values = np.array([12, 10, 20, 15])
weights = np.array([2, 1, 3, 2])
W = 5

F, items = knapsack_bottomup(values, weights, W)

print(F)
for i in range(0,len(values)+1):
    print(items[i])


[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0. 12. 12. 12. 12.]
 [ 0. 10. 12. 22. 22. 22.]
 [ 0. 10. 12. 22. 30. 32.]
 [ 0. 10. 15. 25. 30. 37.]]
[[], [], [], [], [], []]
[[], [], [12], [12], [12], [12]]
[[], [10], [12], [10, 12], [10, 12], [10, 12]]
[[], [10], [12], [10, 12], [20, 10], [20, 12]]
[[], [10], [15], [15, 10], [20, 10], [15, 10, 12]]


In [9]:
values = np.array([12, 10, 20, 15])
weights = np.array([2, 1, 3, 2])
W = 5

# create an array for the subproblem solutions (-1 flags subproblem cells that haven't been computed yet)
F = np.ones(shape=(len(values)+1, W+1))*-1.0
# set base case
F[0,:] = 0
F[:,0] = 0
 
# items picked for each subproblem
items_picked = []
for i in range(0,len(values)+1):
    b = []
    for j in range(0, W+1):
        b.append([])
    items_picked.append(b)

# solve using efficient top down approach
knapsack_topdown_recursive(len(values), W, values, weights, F, items_picked)    
print(F)
for i in range(0,len(values)+1):
    print(items_picked[i])

[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0. 12. 12. 12. 12.]
 [ 0. -1. 12. 22. -1. 22.]
 [ 0. -1. -1. 22. -1. 32.]
 [ 0. -1. -1. -1. -1. 37.]]
[[], [], [], [], [], []]
[[], [], [12], [12], [12], [12]]
[[], [], [12], [10, 12], [], [10, 12]]
[[], [], [], [10, 12], [], [20, 12]]
[[], [], [], [], [], [15, 10, 12]]


Using `Depth-First-Search` to compute the **`Transitive Closure`** of a directed graph with adjacency matrix representation

In [12]:
# computes traisitve eclosure matrix using DFS
def transitive_closure_dfs(A):

    # create a transitive closure matrix and fill with zeros
    T = np.zeros_like(A)

    # perform DFS starting at each vertex
    for v in range(len(A)):

        # mark all vertices as unvisited (0 means unvisited and >0 for the order in which a vertex is visited)
        visited = np.zeros(shape=(len(A)))
        visiting_order = np.zeros(shape=(1))
        dfs_recurse(A, v, visited, visiting_order)

        #print(f"Vertex: {v}, visited: {visited}")

        # fill the T matrix row corresponding to vertex v
        for w in range(len(A)):
            if(visited[w] > 0):
                T[v,w] = 1             

    return T

# performs recursive depth first search on a graph with adjacency matrix A starting from vertex number v
def dfs_recurse(A, v, visited, visiting_order):

    visiting_order[0] += 1
    
    # recursively traverse to all unvisited neighboring vertices `w` of vertex `v`   
    for w in range(len(A)):
        if((A[v,w] == 1) and visited[w] == 0):
            # mark w as visited and traverse to w        
            visited[w] = visiting_order[0] 
            dfs_recurse(A, w, visited, visiting_order) 

In [13]:
# example adjacency matrix of a directed graph with 4 vertices, numbered 0,1,2,3
A = np.array([[0,1,0,1], 
              [0,0,0,1], 
              [0,0,0,0],
              [1,0,1,0]])

T = transitive_closure_dfs(A)

print(T)

[[1 1 1 1]
 [1 1 1 1]
 [0 0 0 0]
 [1 1 1 1]]


Using `Bredth-First-Search` to compute the **`Transitive Closure`** of a directed graph with adjacency matrix representation

In [17]:
# computes traisitve eclosure matrix using DFS
def transitive_closure_bfs(A):

    # create a transitive closure matrix and fill with zeros
    T = np.zeros_like(A)

    # perform BFS starting at each vertex
    for v in range(len(A)):

        # mark all vertices as unvisited (0 means unvisited and >0 for the order in which a vertex is visited)
        visited = np.zeros(shape=(len(A)))
        bfs(A, v, visited)

        #print(f"Vertex: {v}, visited: {visited}")

        # fill the T matrix row corresponding to vertex v
        for w in range(len(A)):
            if(visited[w] > 0):
                T[v,w] = 1             

    return T

# performs bredth first search on a graph with adjacency matrix A starting from vertex number v
def bfs(A, v, visited):

    # create a queue and load vertex v onto 
    bfs_queue = [v]
    visiting_order = 1
        
    # stop when queue becomes empty
    while (len(bfs_queue) > 0):  
        # select the front element of the queue
        front = bfs_queue[0] 
        # visit each unvisited neighbor 'w' of the vertex at the front of the queue
        for w in range(len(A)):
            if((A[front,w] == 1) and visited[w] == 0):
                # mark w as visited and push it onto queue     
                visited[w] = visiting_order
                bfs_queue.append(w)
                visiting_order += 1
        # after visiting all unvisited neighbors of the front element, pop out the front element and repeat the process with new front element
        bfs_queue.pop(0)        

In [18]:
# example adjacency matrix of a directed graph with 4 vertices, numbered 0,1,2,3
A = np.array([[0,1,0,1], 
              [0,0,0,1], 
              [0,0,0,0],
              [1,0,1,0]])

T = transitive_closure_bfs(A)

print(T)

[[1 1 1 1]
 [1 1 1 1]
 [0 0 0 0]
 [1 1 1 1]]
